In [2]:
#Goals:
#test usability of custom Python functions in tf.Operator
    #outcome: technically yes, but limited - tensorflow uses py functions statically
    #seems unsuitable for udp
#are Python library methods allowed?
    #also yes, practically no
#what about runtimes from other languages?
    #not tested yet, probably same as python if wrapped into py lib
#what about arbitrary runtimes?
    #seems ok so far, need to actively test docker runtime and see if tf graph will work with modded udp layer

#need some kind of clock function to keep everything synchronous

In [2]:
import tensorflow as tf
from tensorflow.keras import Model, activations, initializers
from tensorflow.keras.layers import Dense, Input
from multiprocessing import Process
import numpy as np
import time
import socket, sys
from sympy import *

In [3]:
tf.executing_eagerly()
tf.__version__

'2.3.1'

In [5]:
#making a tf function:
@tf.function
def xor(x,y):
    return x ^ y
tf.print(xor(0,1))

1


In [6]:
#with a library:
#example from sympy documentation
#link
#https://docs.sympy.org/latest/tutorial/basic_operations.html

@tf.function
def floating_point_sqrt(x):
    expr = sqrt(x)
    return tf.convert_to_tensor(value=expr.evalf(), dtype=tf.float64)
tf.print(floating_point_sqrt(2))

1.4142135623730951


In [1]:
#arbitrary runtimes
#using docker container
#map post-activation (binary step function, relu, or leaky relu) data values to individual functions
#1 function per container
#alpine container
#put all containers into a tensorflow function
#tensorflow function implements port mapping
#docker container reads from ports, processes, and returns data

#assume the containers are already running

In [38]:
#use tensorflow layer as client, container as server

#make function able to send any piece of a tensor to a port
#run data transmissions (tensor elements -> container) in parallel
    #get tensor, split up into pieces
    #process for each thread: send to container, given socket, data point, host and port

def send_to_container(pre_activation_tensor, ports_list, host):
    #test tensor for checking dtype
    string_tensor = tf.constant("samplestring")
    # create dgram udp socket
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    except socket.error:
	    print('Failed to create socket')
	    sys.exit()

    #add exception to avoid nested strings
    if pre_activation_tensor.dtype == string_tensor.dtype:
        msg = tf.get_static_value(pre_activation_tensor)
    else:
        msg = str.encode(str(tf.get_static_value(pre_activation_tensor)))

    for port in ports_list:
        s.sendto(msg, (host, port))
    return 0

#not yet compatible with strings - would need character encoding/decoding on front end
#send_to_container(pre_activation_tensor=tf.constant("bruh"), ports_list=[50007], host='localhost')
send_to_container(pre_activation_tensor=tf.constant([1,2,1,4]), ports_list=[1234], host='localhost')

0

In [12]:
#Use UDP client in custom Keras layer
#Code modified from reference: https://keras.io/guides/making_new_layers_and_models_via_subclassing/

class Linear_Binary_UDP(tf.keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        #start with all the connections on
        w_init = initializers.Ones()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = initializers.Zeros()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

#include activation functions, but after layer return
#worry about activations when building network on Keras
#may need separate activation layers (see keras docs)
    def call(self, inputs):
        tf.matmul(inputs, self.w) + self.b
        #workspot - put 
        return 


In [36]:
generic_tensor = tf.constant("bruh")
generic_number_tensor = tf.constant([1,2,3,4])
print(type(tf.get_static_value(generic_tensor)))

string_tensor = tf.constant("test string")
bytes_tensor = tf.constant(b'bruh')

print((generic_tensor.dtype) == (string_tensor.dtype))
print(generic_tensor.dtype)
print(string_tensor.dtype)
print(bytes_tensor.dtype)
print(type(generic_number_tensor))

<class 'bytes'>
True
<dtype: 'string'>
<dtype: 'string'>
<dtype: 'string'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [3]:
#standard dense feedforward netowrk
input_layer = Input(shape=(4,))

linear1 = Dense(16,activation='sigmoid')(input_layer)
linear2 = Dense(16,activation='sigmoid')(linear1)
linear3 = Dense(16,activation='sigmoid')(linear2)
linear4 = Dense(16,activation='sigmoid')(linear3)

output = Dense(1,activation='sigmoid')(linear4)

dense_model = tf.keras.Model(inputs=input_layer, outputs=output)
dense_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                80        
_________________________________________________________________
dense_6 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_7 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_8 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 17        
Total params: 913
Trainable params: 913
Non-trainable params: 0
________________________________________________________

In [2]:
#feedforward network with concatenation
input_layer = Input(shape=(4,))

linear1 = Dense(16,activation='sigmoid')(input_layer)
linear2 = Dense(16,activation='sigmoid')(linear1)
linear3 = Dense(16,activation='sigmoid')(linear2)
linear4 = Dense(16,activation='sigmoid')(linear3)

#layers also map straight from each sequential layer to the output
concatenate = tf.keras.layers.Concatenate()([input_layer, linear1, linear2, linear3, linear4])
output = Dense(1,activation='sigmoid')(concatenate)

dag_model = tf.keras.Model(inputs=input_layer, outputs=output)
dag_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 16)           80          input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 16)           272         dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 16)           272         dense_1[0][0]                    
_______________________________________________________________________________________